In [2]:
#!/usr/bin/env python
# coding: utf-8
import argparse
import os
import sys
from collections import Counter, defaultdict
from copy import deepcopy
import datetime
import random
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.stats import zscore
import networkx as nx
import igraph as ig
import community
import leidenalg as la
from node2vec import Node2Vec
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.manifold import TSNE
import matplotlib.cm as cm
import itertools
import csv
from umap import UMAP
import time
from cdlib import algorithms
import seaborn as sns
import matplotlib.pyplot as plt


from Simulation_functions import *
from CoMemDIPHW import *
from DIPHW import *
from plot_functions import *

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
from functions import *





folder='ngenes'
filename = f'ClusteringPerformance_{folder}_parallel.csv'




rows=3000
cols=1000
num_modules=30
avg_genes_per_module = 35
avg_cells_per_module=38
target_density= 0.03
module_density=0.3
inter_module_density=0.1
inter_module_connection_probability=0.6
lambda_background=10
lambda_module=20
inter_module_lambda=10








# # 1.1 Generate a Simulated Sparse Matrix (count data)
# # 1.2 Normalise the sparse counts data
# # 2.  Correlation-Based Graph Projection




modules = simulate_input_modules(rows, cols, num_modules, avg_genes_per_module,avg_cells_per_module)

print("Generated Modules:")
for i, (start_row, end_row, start_col, end_col) in enumerate(modules):
    print(f"Module {i + 1}: Rows {start_row}-{end_row}, Columns {start_col}-{end_col}")

sparse_data = create_sparse_matrix_with_inter_module_variance(
    rows, cols, modules, target_density, module_density, inter_module_density, 
    inter_module_connection_probability, lambda_background, lambda_module, inter_module_lambda
)




data_df_community = pd.DataFrame.sparse.from_spmatrix(sparse_data.T)
community_assignments=modules_to_community_dict(modules, cols)
data_df_community['community']=list(community_assignments.values())
data_df_community=data_df_community.T
print(sum([i=='NA' for i in list(community_assignments.values())]))




column_names = data_df_community.columns.tolist()

shuffled_column_names = np.random.permutation(column_names)

shuffled_df_community = data_df_community[shuffled_column_names]

shuffled_sparse_data=sp.csr_matrix(shuffled_df_community.iloc[:-1].astype(np.float64))



# # Shuffle the simulated scRNAseq matrix
# # G_weighted_HT is a weighted graph but sparsified by HT
# # G_weighted is a fully-connected weighted graph without sparsification




def normalize_sparsematrix_cpm(matrix):
    """
    Normalize the matrix using Counts Per Million (CPM) normalization.

    :param matrix: A sparse matrix
    :return: Normalized matrix
    """
    dense_matrix = matrix.toarray()
    column_sums = np.sum(dense_matrix, axis=0)
    column_sums[column_sums == 0] = 1
    normalized_matrix = 1e6 * dense_matrix / column_sums
    return sp.csr_matrix(normalized_matrix)



def normalize_matrix_cpm(matrix):
    """
    Normalize the matrix using Counts Per Million (CPM) normalization.

    :param matrix: A sparse matrix
    :return: Normalized matrix
    """
    matrix=matrix.copy()
    column_sums = np.sum(matrix, axis=0)
    column_sums[column_sums == 0] = 1
    normalized_matrix = 1e6 * matrix / column_sums
    return normalized_matrix

def filter_zero_expression_community(expression_df_with_CommunityAssignment,thres=0):
    """
    Input: expression dataframe with the last row being the community assignment
    Filter out genes (rows) and cells (columns) with zero expression.
    :return: filtered expression dataframe with community assignment
    """
    df=expression_df_with_CommunityAssignment

    last_row = df.iloc[-1, :]

    row_sums = df.iloc[:-1, :].sum(axis=1)
    col_sums = df.iloc[:-1, :].sum(axis=0)

    filtered_genes = row_sums > thres
    filtered_df = df.iloc[:-1, :].loc[filtered_genes, :]

    filtered_cells = col_sums > thres
    filtered_df = filtered_df.loc[:, filtered_cells]
    last_row_filtered=last_row.loc[filtered_cells]
    filtered_df = filtered_df.append(last_row_filtered)

    return filtered_df



def generate_custom_colormap(num_colors):
    """
    Generate a custom colormap with a specified number of colors by combining multiple Sequential Color Brewer palettes.
    """
    colormaps = ['RdPu', 'Purples_d', 'flare','Wistia','Greens_d','Oranges_d','Blues','gray']  
    
    
    #colormaps = ['Spectral'] 
    n_maps=len(colormaps)
    avg_c_per_palette=int(np.floor(num_colors/n_maps))
    residual=num_colors%n_maps
    n_c_per_palette=[avg_c_per_palette for i in range(n_maps)]
    
    for i,j in enumerate(range(residual)):
        n_c_per_palette[i]+=1
        
    
    color_list = []

    while len(color_list) < num_colors:
        for i,cmap in enumerate(colormaps):
            colors = sns.color_palette(cmap, n_colors=n_c_per_palette[i])
            color_list.extend(colors)

    color_list = color_list[:num_colors]

    return color_list






/home/he.wan1/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-01 23:25:46.781430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-01 23:25:47.478767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/anaconda3/2022.05/lib:/shared/centos7/nodejs/14.15.4/lib
2025-01-01 23:25:47.478831: I tensorflow/compiler/xla/stream

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'graph_tool', 'bayanpy', 'infomap'}


Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}
Generated Modules:
Module 1: Rows 0-35, Columns 0-33
Module 2: Rows 100-130, Columns 33-63
Module 3: Rows 200-235, Columns 66-100
Module 4: Rows 300-338, Columns 100-133
Module 5: Rows 400-442, Columns 133-166
Module 6: Rows 500-527, Columns 166-200
Module 7: Rows 600-635, Columns 200-229
Module 8: Rows 700-744, Columns 233-259
Module 9: Rows 800-833, Columns 266-298
Module 10: Rows 900-955, Columns 300-333
Module 11: Rows 1000-1044, Columns 333-366
Module 12: Rows 1100-1127, Columns 366-400
Module 13: Rows 1200-1232, Columns 400-433
Module 14: Rows 1300-1334, Columns 433-466
Module 15: Rows 1400-1440, Columns 466-493
Module 16: Rows 1500-1524, Columns 500-533
Module 17: Rows 1600-1643, Columns 533-566
Module 18: Rows 1700-1732, Columns 566-600
Module 19: Ro

In [4]:

num_clusters = num_modules 
num_colors=num_clusters
color_palette = generate_custom_colormap(num_colors)




non_zero_df = filter_zero_expression_community(shuffled_df_community)
non_zero_data=np.array(non_zero_df[:-1].values)
print(np.shape(non_zero_data))
community_assignments=dict(zip(non_zero_df.columns,non_zero_df.loc['community']))
normalized_data = normalize_matrix_cpm(non_zero_data).astype(float)


logtransform=True


if logtransform==True:
    preprocessed_data=np.log2(normalized_data+1)

else:
    preprocessed_data=normalized_data

preprocessed_df=non_zero_df.copy()
nrows,ncols=preprocessed_df.shape
preprocessed_df.iloc[:-1,:ncols]=preprocessed_data
node_list=list(preprocessed_df.columns)

preprocessed_df

(3000, 1000)


,377,2,37,211,114,163,549,726,723,341,...,708,845,681,685,116,566,435,306,507,738
0,0.0,0.0,0.0,0.0,0.0,13.124375,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.678178,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,12.998859,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.941175,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.163541,0.0,12.980879,...,0.0,0.0,0.0,0.0,0.0,0.0,12.871966,0.0,0.0,0.0
3,0.0,13.756985,0.0,0.0,0.0,0.0,0.0,12.4632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.256263,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import Simulation_functions
import importlib

importlib.reload(Simulation_functions)

from Simulation_functions import *

In [6]:
correlation_threshold_percentile=99

# Using Pearson correlation (default)
cell_coexpression_weighted_HT_pearson, cell_coexpression_weighted_pearson = create_correlation_network(preprocessed_data, threshold_percentile=99, method='pearson')
gene_coexpression_weighted_HT_pearson, gene_coexpression_weighted_pearson = create_correlation_network(preprocessed_data.T, threshold_percentile=99, method='pearson')

# Using Spearman correlation
cell_coexpression_weighted_HT_spearman, cell_coexpression_weighted_spearman = create_correlation_network(preprocessed_data, threshold_percentile=99, method='spearman')
gene_coexpression_weighted_HT_spearman, gene_coexpression_weighted_spearman = create_correlation_network(preprocessed_data.T, threshold_percentile=99, method='spearman')

# Using Cosine similarity
cell_coexpression_weighted_HT_cosine, cell_coexpression_weighted_cosine = create_correlation_network(preprocessed_data, threshold_percentile=99, method='cosine')
gene_coexpression_weighted_HT_cosine, gene_coexpression_weighted_cosine = create_correlation_network(preprocessed_data.T, threshold_percentile=99, method='cosine')


# Using Kendall's tau 
cell_coexpression_weighted_HT_kendall, cell_coexpression_weighted_kendall = create_correlation_network(preprocessed_data, threshold_percentile=99, method='kendall')
gene_coexpression_weighted_HT_kendall, gene_coexpression_weighted_kendall = create_correlation_network(preprocessed_data.T, threshold_percentile=99, method='kendall')


passed


In [7]:
cell_coexpression_weighted_HT_kendall, cell_coexpression_weighted_kendall = create_correlation_network(preprocessed_data, threshold_percentile=99, method='kendall')
gene_coexpression_weighted_HT_kendall, gene_coexpression_weighted_kendall = create_correlation_network(preprocessed_data.T, threshold_percentile=99, method='kendall')


In [8]:

def create_correlation_network(matrix, threshold_percentile=99, method='pearson'):
    # assume matrix columns are the cells
    if method == 'pearson':
        cell_coexpression = abs(np.corrcoef(matrix.T))
    elif method == 'spearman':
        corr, _ = spearmanr(matrix)
        cell_coexpression = abs(corr)
    elif method == 'cosine':
        distance_matrix = pdist(matrix.T, metric='cosine')
        cell_coexpression = 1 - squareform(distance_matrix)
    elif method == 'kendall':
        def compute_kendall(i, j):
            tau, _ = kendalltau(matrix[:, i], matrix[:, j])
            return tau

        num_cells = matrix.shape[1]
        cell_coexpression = np.zeros((num_cells, num_cells))

        results = Parallel(n_jobs=-1)(
            delayed(compute_kendall)(i, j) for i in range(num_cells) for j in range(i, num_cells)
        )

        index = 0
        for i in range(num_cells):
            for j in range(i, num_cells):
                tau = results[index]
                cell_coexpression[i, j] = abs(tau)
                cell_coexpression[j, i] = abs(tau)
                index += 1

    else:
        raise ValueError(f"Unsupported correlation method: {method}")

    cell_coexpression_flat = cell_coexpression.flatten()
    cell_coexpression_weighted = deepcopy(cell_coexpression)

    threshold = np.percentile(cell_coexpression_flat, threshold_percentile)
    cell_coexpression[cell_coexpression < threshold] = 0
    cell_coexpression_filtered = cell_coexpression

    return cell_coexpression_filtered, cell_coexpression_weighted



In [9]:
for correlation_computation_method in ['spearman','cosine','pearson','kendall']:
#for correlation_computation_method in ['kendall']:

    
    exec(f'G_cell_weighted = nx.from_numpy_array(cell_coexpression_weighted_{correlation_computation_method})')
    exec(f'G_cell_weighted_HT = nx.from_numpy_array(cell_coexpression_weighted_HT_{correlation_computation_method})')
    exec(f'clustering_performance_{correlation_computation_method}=defaultdict()')

    cell_map={i:j for i,j in enumerate(list(preprocessed_df.columns))}
    G_cell_weighted = nx.relabel_nodes(G_cell_weighted, cell_map)
    G_cell_weighted_HT = nx.relabel_nodes(G_cell_weighted_HT, cell_map)

    print(len(G_cell_weighted.nodes))

    print(list(G_cell_weighted.nodes)==node_list)





    density = nx.density(G_cell_weighted_HT)
    print("Density of the weighted graph after HT:", density)

    density = nx.density(G_cell_weighted)
    print("Density of the weighted graph without HT:", density)

    non_zero_weights = [data['weight'] for _, _, data in G_cell_weighted_HT.edges(data=True) if data['weight'] > 0]

    if non_zero_weights:  
        min_weight = min(non_zero_weights)
        print("Minimum non-zero edge weight:", min_weight)

    print(np.sum(np.sum(non_zero_data,axis=0)==1))
    print(np.sum(np.sum(non_zero_data,axis=0)==0))
    print(np.sum(np.sum(normalized_data,axis=0)==1))
    print(np.sum(np.sum(normalized_data,axis=0)==0))
    print(np.sum(np.sum(normalized_data,axis=0)))
    print(np.sum(np.sum(normalized_data,axis=0)))




    # Define the size threshold
    size_threshold = 5

    community_sizes = Counter(community_assignments.values())

    filtered_communities = {node: community_id for node, community_id in community_assignments.items() if community_sizes[community_id] >= size_threshold}

    organized_communities = defaultdict(list)

    for node, community_id in filtered_communities.items():
        organized_communities[community_id].append(node)

    organized_communities = dict(organized_communities)

    edges_with_weights_HT = [(u, v, d['weight']) for u, v, d in G_cell_weighted_HT.edges(data=True)]
    print(edges_with_weights_HT[:10])


    # # correlation after hard thresholding percentile preprocessed_data_array





    print(np.sum(np.sum(non_zero_data,axis=0)==1))
    print(np.sum(np.sum(non_zero_data,axis=0)==0))

    print(np.sum(np.sum(preprocessed_data,axis=0)==1))
    print(np.sum(np.sum(preprocessed_data,axis=0)==0))


    print(np.sum(np.sum(preprocessed_data,axis=0)))
    print(np.sum(np.sum(preprocessed_data,axis=0)))

    data=preprocessed_data



    # # correlation network

    # # clustering by community detection on correlation networks



    print(preprocessed_data.shape)
    print(sparse_data.shape)




    import community
    G=G_cell_weighted

    partition = community.best_partition(G, weight='weight')



    size_threshold = 5

    community_sizes = Counter(partition.values())

    # Filter communities based on the size threshold
    filtered_communities = {node: community_id for node, community_id in partition.items() if community_sizes[community_id] >= size_threshold}

    organized_communities = defaultdict(list)
    for node, community_id in filtered_communities.items():
        organized_communities[community_id].append(node)
    organized_communities = dict(organized_communities)



    df = preprocessed_df.T.copy()


    df['community'] = list(partition.values())

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)







    community_dict=partition
    sorted_dict = {k: community_dict[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")




    exec(f"clustering_performance_{correlation_computation_method}['Louvain_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['Louvain_weighted_NMI']=nmi_score")





    # # Girvan Newman doesnt scale well way too slow
    # 




    #communities = next(girvan_newman(G))
    #community_dict = {node: i for i, community in enumerate(communities) for node in community}
    #sorted_dict = {k: community_dict[k] for k in node_list}
    #predicted_labels = list(sorted_dict.values())


    # # Leiden



    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)


    partition_leiden = la.find_partition(
        G_ig, 
        la.RBConfigurationVertexPartition,  
        weights=[i['weight'] for i in G_ig.es['weight']], resolution_parameter=1.1,n_iterations=-1)

    community_dict_leiden = {node: cid for cid, community in enumerate(partition_leiden) for node in community}
    sorted_dict = {k: community_dict_leiden[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['Leiden_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['Leiden_weighted_NMI']=nmi_score")




    print(len(set(predicted_labels)))




    df = preprocessed_df.T.copy()


    df['community'] = list(community_dict_leiden.values())

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)





    # # Clauset-Newman-Moore greedy modularity
    # https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html
    # 
    # References
    # 
    # [1]
    # Newman, M. E. J. “Networks: An Introduction”, page 224 Oxford University Press 2011.
    # [2]
    # Clauset, A., Newman, M. E., & Moore, C. “Finding community structure in very large networks.” Physical Review E 70(6), 2004.
    # [3]
    # Reichardt and Bornholdt “Statistical Mechanics of Community Detection” Phys. Rev. E74, 2006.
    # [4]
    # Newman, M. E. J.”Analysis of weighted networks” Physical Review E 70(5 Pt 2):056131, 2004.





    partition_greedy_modularity = nx.community.greedy_modularity_communities(G,weight='weight')

    community_dict_gm={i:n for n, c in enumerate(partition_greedy_modularity) for i in c}


    sorted_dict = {k: community_dict_gm[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['GreedyModularity_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['GreedyModularity_weighted_NMI']=nmi_score")






    df = preprocessed_df.T.copy()


    df['community'] = [community_dict_gm[i] for i in node_list]

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)







    # # Community detection HT


    # weighted but sparsified by HT
    G=G_cell_weighted_HT

    partition = community.best_partition(G, weight='weight')
    size_threshold = 5
    community_sizes = Counter(partition.values())
    filtered_communities = {node: community_id for node, community_id in partition.items() if community_sizes[community_id] >= size_threshold}

    organized_communities = defaultdict(list)
    for node, community_id in filtered_communities.items():
        organized_communities[community_id].append(node)
    organized_communities = dict(organized_communities)

    community_dict=partition
    sorted_dict = {k: community_dict[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')
    print("community_detection_weightedHT")
    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")


    exec(f"clustering_performance_{correlation_computation_method}['Louvain_weightedHT_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['Louvain_weightedHT_NMI']=nmi_score")


    # # Infomap

    G=G_cell_weighted
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_infomap(edge_weights=[i['weight'] for i in G_ig.es['weight']])
    community_dict_infomap = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_infomap.values())))




    sorted_dict = {k: community_dict_infomap[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['infomap_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['infomap_weighted_NMI']=nmi_score")



    G=G_cell_weighted_HT
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_infomap(edge_weights=[i['weight'] for i in G_ig.es['weight']])
    community_dict_infomap = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_infomap.values())))




    sorted_dict = {k: community_dict_infomap[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")




    exec(f"clustering_performance_{correlation_computation_method}['infomap_weightedHT_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['infomap_weightedHT_NMI']=nmi_score")



    df = preprocessed_df.T.copy()


    df['community'] = list(community_dict_infomap.values())

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)


    method='Infomap'

    # # leading_eigenvector

    G=G_cell_weighted
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_leading_eigenvector(clusters=num_clusters,weights=[i['weight'] for i in G_ig.es['weight']])
    community_dict_eigen = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_eigen.values())))




    sorted_dict = {k: community_dict_eigen[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['eigenvector_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['eigenvector_weighted_NMI']=nmi_score")


    G=G_cell_weighted_HT
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_leading_eigenvector(clusters=num_clusters,weights=[i['weight'] for i in G_ig.es['weight']])
    community_dict_eigen = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_eigen.values())))




    sorted_dict = {k: community_dict_eigen[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['eigenvector_weightedHT_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['eigenvector_weightedHT_NMI']=nmi_score")



    df = preprocessed_df.T.copy()


    df['community'] = list(community_dict_eigen.values())

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)


    method='Eigenvector'



    # # community_multilevel
    G=G_cell_weighted
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_multilevel(weights=[i['weight'] for i in G_ig.es['weight']], resolution=1)
    community_dict_multilevel = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_multilevel.values())))




    sorted_dict = {k: community_dict_multilevel[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")


    exec(f"clustering_performance_{correlation_computation_method}['Multilevel_weighted_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['Multilevel_weighted_NMI']=nmi_score")



    G=G_cell_weighted_HT
    G_ig = ig.Graph.TupleList(G.edges(data=True), weights=True)

    partition = G_ig.community_multilevel(weights=[i['weight'] for i in G_ig.es['weight']], resolution=1)
    community_dict_multilevel = {node: cid for cid, community in enumerate(partition) for node in community}
    print(len(set(community_dict_multilevel.values())))




    sorted_dict = {k: community_dict_multilevel[k] for k in node_list}
    predicted_labels = list(sorted_dict.values())

    community_dict=community_assignments
    sorted_dict = {k: community_dict[k] for k in node_list}
    true_labels = list(sorted_dict.values())

    ari_score = adjusted_rand_score(true_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(true_labels, predicted_labels, average_method='arithmetic')

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")



    exec(f"clustering_performance_{correlation_computation_method}['Multilevel_weightedHT_ARI']=ari_score")
    exec(f"clustering_performance_{correlation_computation_method}['Multilevel_weightedHT_NMI']=nmi_score")




    df = preprocessed_df.T.copy()


    df['community'] = list(community_dict_multilevel.values())

    df_sorted = df.sort_values('community')

    df_sorted = df_sorted.drop('community', axis=1)


    method='multilevel'





1000
True
Density of the weighted graph after HT: 0.011011011011011011
Density of the weighted graph without HT: 1.002002002002002
Minimum non-zero edge weight: 0.07222417202946398
0
0
0
0
1000000000.0
1000000000.0
[(377, 377, 1.0), (377, 391, 0.08719888646022314), (377, 388, 0.09158751607618193), (377, 191, 0.08306061728970689), (377, 786, 0.08055454725264583), (2, 2, 1.0), (2, 164, 0.08874499765079011), (2, 908, 0.07652186603129553), (2, 4, 0.08900075465282867), (2, 902, 0.07865847405179022)]
0
0
0
0
1715290.8187440804
1715290.8187440804
(3000, 1000)
(3000, 1000)
Adjusted Rand Index: 0.47186920746862676
Normalized Mutual Information: 0.7614719739778023
Adjusted Rand Index: -0.002373998344705438
Normalized Mutual Information: 0.11360564548385516
25
Adjusted Rand Index: 0.20341890758296238
Normalized Mutual Information: 0.5221961327541592
community_detection_weightedHT
Adjusted Rand Index: 0.43806168569679826
Normalized Mutual Information: 0.6593057391781157
1
Adjusted Rand Index: 0.0


In [15]:
for i,correlation_computation_method in enumerate(['pearson','cosine','spearman','kendall']):
    exec(f'dict{i+1}=clustering_performance_{correlation_computation_method}')

In [33]:
import pandas as pd
import numpy as np

# Assuming you've already executed the code to create dict1, dict2, dict3, and dict4

# Create a DataFrame from the dictionaries
df = pd.DataFrame({
    "Pearson": dict1,
    "Cosine": dict2,
    "Spearman": dict3,
    "Kendall": dict4
})

# Rename the index for better readability
method_names = {
    'Louvain_weighted': 'Louvain',
    'Leiden_weighted': 'Leiden',
    'GreedyModularity_weighted': 'Greedy Modularity',
    'infomap_weightedHT': 'Infomap',
    'eigenvector_weighted': 'Eigenvector'
}

df.index = [method_names.get(idx.rsplit('_', 1)[0], idx) for idx in df.index]

# Separate ARI and NMI
df_ari = df[df.index.str.endswith('ARI')].copy()
df_nmi = df[df.index.str.endswith('NMI')].copy()

# Remove the metric suffix from the index
df_ari.index = df_ari.index.str.replace('_ARI', '')
df_nmi.index = df_nmi.index.str.replace('_NMI', '')

# Function to format the table
def format_table(df, metric):
    # Round to 3 decimal places
    df = df.round(3)
    
    # Add a row for the mean
    df.loc['Mean'] = df.mean()
    
    # Format as strings with 3 decimal places
    df = df.applymap(lambda x: f"{x:.3f}")
    
    # Bold the lowest value in each row (excluding the mean row)
    for idx in df.index[:-1]:
        min_val = df.loc[idx].astype(float).min()
        df.loc[idx] = df.loc[idx].apply(lambda x: f"\\textbf{{{x}}}" if float(x) == min_val else x)
    
    # Add the metric to the index name
    df.index = [f"{idx} ({metric})" for idx in df.index]
    
    return df

# Format ARI and NMI tables
df_ari_formatted = format_table(df_ari, "ARI")
df_nmi_formatted = format_table(df_nmi, "NMI")

# Combine the formatted tables
df_combined = pd.concat([df_ari_formatted, df_nmi_formatted])

# Generate LaTeX code
latex_code = df_combined.to_latex(escape=False)

print(latex_code)


\begin{tabular}{lllll}
\toprule
{} &         Pearson &          Cosine &         Spearman &         Kendall \\
\midrule
Louvain_weightedHT (ARI)     &           0.495 &           0.450 &            0.438 &  \textbf{0.427} \\
infomap_weighted (ARI)       &  \textbf{0.000} &  \textbf{0.000} &   \textbf{0.000} &  \textbf{0.000} \\
eigenvector_weightedHT (ARI) &           0.002 &  \textbf{0.000} &            0.003 &           0.005 \\
Multilevel_weighted (ARI)    &          -0.001 &          -0.003 &  \textbf{-0.004} &          -0.002 \\
Multilevel_weightedHT (ARI)  &  \textbf{0.032} &           0.034 &            0.034 &           0.035 \\
Mean (ARI)                   &           0.106 &           0.096 &            0.094 &           0.093 \\
Louvain_weightedHT (NMI)     &           0.690 &           0.660 &            0.659 &  \textbf{0.655} \\
infomap_weighted (NMI)       &  \textbf{0.000} &  \textbf{0.000} &   \textbf{0.000} &  \textbf{0.000} \\
eigenvector_weightedHT (NMI) &          